In [ ]:
import altair as alt
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt

from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix
#from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
#from imblearn.over_sampling import RandomOverSampler
from sklearn import metrics
from sklearn import tree

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)
# allows graph to work despite large data set
alt.data_transformers.disable_max_rows()

Data = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv', parse_dates=["date"])

Model_Data = Data
#Model_Data['date'] = Model_Data['date'].dt.strftime('%Y-%m-%d')
Model_Data['bedrooms'] = Model_Data['bedrooms'].astype('int64')
Model_Data['bathrooms'] = Model_Data['bathrooms'].astype('float')
Model_Data['sqft_living'] = Model_Data['sqft_living'].astype('int64')
Model_Data['waterfront'] = Model_Data['waterfront'].astype('category')
Model_Data['view'] = Model_Data['view'].astype('category')
Model_Data['condition'] = Model_Data['condition'].astype('category')
Model_Data['grade'] = Model_Data['grade'].astype('int64')
Model_Data['zipcode'] = Model_Data['zipcode'].astype('category')

In [ ]:
Model_Data.head()

#NOTE: Data is from May 2014 - May 2015, From the King County, Seattle area

Index(['id', 'date', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'price', 'price_log', 'sqft_log',
       'month'],
      dtype='object')

## Data Exploration

In [ ]:
#TODO Number Figures

### Geo Data Exploration

In [ ]:
import geopandas as gpd
from shapely.geometry import Point, Polygon

In [ ]:
kings_county_map = gpd.read_file('data/kc_tract_10.shp')

crs = {'init':'EPSG:4326'}
geometry = [Point(xy) for xy in zip(Model_Data['long'], Model_Data['lat'])]
geo_df = gpd.GeoDataFrame(Model_Data,
                          crs = crs,
                          geometry = geometry)

ERROR:fiona._env:data/kc_tract_10.shp: No such file or directory


DriverError: data/kc_tract_10.shp: No such file or directory

In [ ]:
geo_df['price_log'] = np.log(geo_df['price'])
fig, ax = plt.subplots(figsize = (10,10))
kings_county_map.to_crs(epsg=4326).plot(ax=ax, color='lightgrey')
geo_df.plot(column = 'price_log', ax=ax, cmap = 'rainbow',
            legend = True, legend_kwds={'shrink': 0.3},
            markersize = 10)
ax.set_title('King County Price Heatmap')
plt.savefig('Heat Map')

In [ ]:
geo_df['sqft_log'] = np.log(geo_df['sqft_living'])
fig, ax = plt.subplots(figsize = (10,10))
kings_county_map.to_crs(epsg=4326).plot(ax=ax, color='lightgrey')
geo_df.plot(column = 'sqft_log', ax=ax, cmap = 'winter',
            legend = True, legend_kwds={'shrink': 0.3},
            alpha = .5)
ax.set_title('Sqft Heatmap')
plt.savefig('Sqft Heat Map')

In [ ]:
geo_df['price_log'] = np.log(geo_df['price'])

geo_df['date'] = geo_df['date'].dt.strftime('%Y-%m-%d %H:%M:%S')

geo_df.explore(column = 'price_log', cmap = 'rainbow',
            legend = True, legend_kwds={'shrink': 0.3},
            markersize = 10)

In [ ]:
slider_df = Model_Data

slider_df['price_log'] = np.log(slider_df['price'])
slider_df['sqft_log'] = np.log(slider_df['sqft_living'])

slider_df.sort_values(by='date', inplace=True)

slider_df['month'] = slider_df['date'].to_numpy().astype('datetime64[M]')

#using plotly for an animated choropleth map
import plotly.express as px


In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/wa_washington_zip_codes_geo.min.json') as response:
    zipcodes = json.load(response)

px.scatter_mapbox(
    slider_df,
    lat="lat",
    lon="long",
    color="price_log",
    size='sqft_living',
    animation_frame="month",
    hover_data=['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'price']
).update_layout(
    mapbox={
        "style": "carto-positron",
        "zoom":11,
        "layers": [
                {
                    "source": zipcodes,
                    "below": "traces",
                    "type": "line",
                    "color": "purple",
                    "line": {"width": 1.5},
                }
            ],
        },
        margin={"l": 0, "r": 0, "t": 0, "b": 0}
)

### Other Data Exploration

In [ ]:
alt.Chart(Model_Data).mark_boxplot().encode(
    x = alt.X("grade", title = "Housing Grade"),
    y = alt.Y("price", title = "Housing Prices")
).properties(
    title = 'Distribution of Prices by Grade'
)

In [ ]:
alt.Chart(Model_Data).mark_circle().encode(
    x = alt.X('sqft_basement', title = "Basement Square Feet"),
    y = alt.Y('price', title = "Housing Prices")
).properties(
    title = "Distribution of Prices by Basement Square Feet"
)

In [ ]:
alt.Chart(Model_Data).mark_circle().encode(
    x = alt.X("sqft_above", title = "Square Feet Above Ground Level"),
    y = alt.Y("price", title = "Housing Prices")
).properties(
    title = 'Distribution of Prices by Square Feet Above Ground Level'
)

In [ ]:
alt.Chart(Model_Data).mark_boxplot().encode(
    x = alt.X("bathrooms", title = "Total Bathrooms"),
    y = alt.Y("price", title = "Housing Prices")
).properties(
    title = 'Distribution of Prices by number of Bathrooms'
)

In [ ]:
alt.Chart(Model_Data).mark_boxplot().encode(
    x = alt.X("bedrooms", title = "Total Bedrooms"),
    y = alt.Y("price", title = "Housing Prices")
).properties(
    title = 'Distribution of Prices by Number of Bedrooms'
)

In [ ]:
alt.Chart(Model_Data).mark_boxplot().encode(
    x = alt.X("floors", title = "Total Floors"),
    y = alt.Y("price", title = "Housing Prices")
).properties(
    title = 'Distribution of Prices by Number of Floors'
)

## ML Stuff

### Train

In [ ]:
# drop id, price, maybe date as an identifier if we aren't using, say, month
# Prep for training
# errors = "ignore" helps the holdout to run. It ignores if the column to drop is not found
X = pd.get_dummies(Model_Data).drop(columns = ['price','date','id'], errors = "ignore")
y = Model_Data['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2024)

scaler = MinMaxScaler()
Features_Scale = ['sqft_living','sqft_lot','sqft_above','sqft_basement','sqft_living15','sqft_lot15']

X_train[Features_Scale] = scaler.fit_transform(X_train[Features_Scale])

# Train the model
clf = XGBRegressor(random_state = 2024)
clf.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=2024, ...)

### Evaluate

In [ ]:
# EVALUATE THE MODEL

X_test[Features_Scale] = scaler.transform(X_test[Features_Scale])

y_pred = clf.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

pd.options.display.float_format = '{:.2f}'.format
# Create a DataFrame to display predicted and true values
results = pd.DataFrame({'True Values': y_test, 'Predicted Values': y_pred})

# Display DataFrame with MSE
print("Mean Squared Error:", mse)
display(results)

Mean Squared Error: 21110638057.282227


,True Values,Predicted Values
6399,423000.00,442115.97
7939,375000.00,388083.66
18162,599000.00,721500.25
4499,310000.00,252692.92
9005,774888.00,556093.38
...,...,...
17860,306950.00,364577.62
15397,855000.00,598033.56
18488,1450000.00,1268090.00
18188,330000.00,271842.38


In [ ]:
# Calculate sum of squares error (SSE)
sse = np.sum((y_test - y_pred)**2)

# Calculate mean squared error (MSE)
mse = mean_squared_error(y_test, y_pred)

# Calculate root mean squared error (RMSE)
rmse = np.sqrt(mse)

# Calculate R² value
r2 = r2_score(y_test, y_pred)

# Print SSE, MSE, RMSE, and R² value
print("Sum of Squares Error (SSE):", sse)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R² Value:", r2)

Sum of Squares Error (SSE): 84442552229128.9
Mean Squared Error (MSE): 21110638057.282227
Root Mean Squared Error (RMSE): 145295.0035523666
R² Value: 0.8553797981993465


### Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

param_grid ={
    'learning_rate':            [0,1],            #1
    'min_split_loss':           range(0, 50, 10), #40, 0
    #'max_depth':                range(0, 6), #10
    #"I would leave max. depth at default (=6). Deeper trees tend to overfit soon. Shallow trees may not learn properly"
    #'min_child_weight':         range(0, 100, 10),
    #'max_delta_step':           range(0, 100, 10),

    #'subsample':                [0,1],
    #'reg_lambda':               range(0, 100),
    #'reg_alpha':                range(0, 100),
    #'auto':                     ['auto', 'exact', 'approx', 'hist'],
    #'updater':                  ['grow_colmaker','grow_histmaker','grow_quantile_histmaker',
    #                             'grow_gpu_hist','grow_gpu_approx','sync','refresh','prune'],
    #'refresh_leaf':             [0,1],
    #'process_type':             ['default', 'update'],
    #'grow_policy':              ['depthwise', 'lossguide'],
    #'max_leaves':               range(0, 100),
    #'max_bin':                  range(0, 2560),
    #'num_parallel_tree':        range(0, 100),
    #'multi_strategy':           ['one_output_per_tree', 'multi_output_tree'],
}

X = pd.get_dummies(Model_Data, drop_first=True).drop(columns = ['price','date','id'], errors = "ignore")
y = Model_Data['price']

X_train, X_other, y_train, y_other = train_test_split(X, y, test_size=0.30, random_state = 42)
X_test, X_val, y_test, y_val = train_test_split(X_other, y_other, test_size=0.50, random_state = 42)

grid_search = GridSearchCV(XGBRegressor(), param_grid, refit = True, verbose = 3,n_jobs=-1)

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
grid_predictions = grid_search.predict(X_test)
#print(grid_search.score(grid_predictions, y_test))



## Holdout

### Prep

In [ ]:
Holdout_Data = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing_holdout_test.csv')

In [ ]:
### Cleanup
Holdout_Data['date'] = pd.to_datetime(Data['date'], format='%Y%m%dT%H%M%S')

Holdout_Data['bedrooms'] = Holdout_Data['bedrooms'].astype('category')
Holdout_Data['bathrooms'] = Holdout_Data['bathrooms'].astype('category')
Holdout_Data['sqft_living'] = Holdout_Data['sqft_living'].astype('int64')
Holdout_Data['waterfront'] = Holdout_Data['waterfront'].astype('category')
Holdout_Data['view'] = Holdout_Data['view'].astype('category')
Holdout_Data['condition'] = Holdout_Data['condition'].astype('category')
Holdout_Data['grade'] = Holdout_Data['grade'].astype('category')
Holdout_Data['zipcode'] = Holdout_Data['zipcode'].astype('category')
Holdout_Data = Holdout_Data.drop(columns = ['date','id','lat','long'])

Holdout_Data = pd.get_dummies(Holdout_Data)

In [ ]:
Holdout_X = Holdout_Data.drop(columns = ['price','date','id'], errors = "ignore")

# Size the holdout to match the model
X, Holdout_X = X.align(Holdout_X, fill_value=0, axis=1, join="left")

# Check to make sure it worked
# It works if the columns are the same (second value should match)
display(X.shape, Holdout_X.shape)

# Scale holdout data
Holdout_X[Features_Scale] = scaler.transform(Holdout_X[Features_Scale])

(20000, 96)

(1613, 96)

### Predict

In [ ]:
# Use the trained model to export prediction data frame with a single column
holdout_prediction = pd.DataFrame({
    "price": clf.predict(Holdout_X)
})

In [ ]:
expected_features = len(clf.get_booster().feature_names)
actual_features = Holdout_Data.columns.shape

print("Expected Features:", expected_features)
print("Actual Features:", actual_features)

Expected Features: 96
Actual Features: (131,)


### Download Predicion CSV

In [ ]:
from google.colab import files

filename = r'team5-module3-predictions.csv' #NOTE FILL IN WITH OWN FILE NAME

try:
    holdout_prediction.to_csv(filename, index=False)
    print("CSV file saved successfully.")
except Exception as e:
    print("An error occurred while saving the CSV file:", e)

files.download(filename)

CSV file saved successfully.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>